In [ ]:
import os
import asyncio
import requests
from dotenv import load_dotenv
from agents import Agent, Runner, trace, SQLiteSession, function_tool

In [ ]:
load_dotenv(override=True)

In [6]:
topic_agent_instructions = "You are a YouTube Channel Topic Generator Agent. You will need to generate a topic that is relevant to the channel and that is not already in the channel's video titles."
topic_agent = Agent(name="topic_agent", instructions=topic_agent_instructions)

planner_agent_instructions = "You are a YouTube Channel Planner Agent. You plan YouTube videos for a channel. You will be given a topic and you will need to plan a video for that topic by breaking the topic into short actionable steps."
planner_agent = Agent(name="planner_agent", instructions=planner_agent_instructions)

script_agent_instructions = "You are a YouTube Channel Script Writer Agent. You will be given a topic. You will need to write an engaging script for a video for that topic and format."
script_agent = Agent(name="script_agent", instructions=script_agent_instructions)

seo_agent_instructions = "You are a YouTube Channel SEO Agent. You will be give a topic and you will need to write a SEO friendly title, description, tags, thumbnails, and a thumbnail description for that topic."
seo_agent = Agent(name="seo_agent", instructions=seo_agent_instructions)

In [7]:
channel = "Startup Building"

In [10]:
@function_tool
def send_mailgun_email_notification(subject: str, body: str) -> str:
    """
    Sends an email using Mailgun with the given subject and body.
    
    Args:
        subject: The email subject line
        body: The email body content
        
    Returns:
        A string indicating success or failure of the email send operation
    """
    response = requests.post(
        f"https://api.mailgun.net/v3/{os.getenv('MAIL_GUN_DOMAIN')}/messages",
        auth=("api", os.getenv('MAIL_GUN_API_KEY')),
        data={
            "from": os.getenv('MAIL_GUN_YOUTUBE_CHANNEL_FROM_EMAIL'),
            "to": os.getenv('MAIL_GUN_YOUTUBE_CHANNEL_TO_EMAIL'),
            "subject": subject,
            "text": body
        }
    )

    if response.status_code == 200:
        return "Email sent successfully."
    else:
        return f"Email failed: {response.text}"

In [11]:
topic_agent_as_tool = topic_agent.as_tool(tool_name="topic_agent_tool", tool_description="A tool that generates a topic for a YouTube video")
planner_agent_as_tool = planner_agent.as_tool(tool_name="planner_agent_tool", tool_description="A tool that plans a YouTube video")
script_agent_as_tool = script_agent.as_tool(tool_name="script_agent_tool", tool_description="A tool that writes a script for a YouTube video")
seo_agent_as_tool = seo_agent.as_tool(tool_name="seo_agent_tool", tool_description="A tool that writes a SEO friendly title, description, tags, thumbnails, and a thumbnail description for a YouTube video")

utility_tools = [topic_agent_as_tool, planner_agent_as_tool, script_agent_as_tool, seo_agent_as_tool]

In [14]:
email_notification_agent_instructions = """
You are a YouTube Channel Email Agent. 
You are responsible for sending email notifications. 
Use the send_mailgun_email_notification tool."""

# email_tools = [send_mailgun_email_notification]
email_notification_agent = Agent(name="email_notification_agent", instructions=email_notification_agent_instructions, tools=[send_mailgun_email_notification], handoff_description="Send an email to the channel collaborators about the video plan, script, publishing events, and SEO.")

In [21]:
channel_manager_agent_instructions = """
You are a YouTube Channel Manager Agent.

You will be given a video topic for a YouTube channel. Your responsibilities:

1. Generate a topic for a YouTube video.
2. Plan the video by breaking the topic into clear, actionable steps.
3. Write an engaging, high-retention script tailored to the format.
4. Generate SEO assets: a title, description, tags, and thumbnail with description.
5. You must send an email to the channel collaborators summarizing the video plan, script, publishing timeline, and SEO assets.

To send an email: 
1. When you're ready to send the email, DO NOT describe it or say you're about to send it. 
    Instead, immediately call the email_notification_agent agent via handoffs with parameters:
    - `subject`: string
    - `body`: string

    You must end your run with this handoff — no final summary needed.
2. Do not print or include the email in your final response.  
3. Do not ask for permission to send it.  
4. Just trigger the handoff with the final subject and body.

You must only call each tool only ONCE. Do not repeat tool usage. Once you get the result from a tool, use that result to proceed. Do not call the same tool again.
"""


channel_manager_agent = Agent(name="channel_manager_agent", instructions=channel_manager_agent_instructions, tools=utility_tools, handoffs=[email_notification_agent])

In [18]:
channel_session = SQLiteSession("channel_session")

In [ ]:
with trace("Automated YouTube Channel Management"):
    result = await Runner.run(channel_manager_agent, channel, session=channel_session)

print(f"Channel Manager Agent Output: {result.final_output}")

# FINALLY received an email!!!

# Each tool was called only once!!!